In [1]:
import pystac_client
import planetary_computer as pc

#import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import shapely
import os
from shapely.geometry import box

# Find items of all NAIP scenes that intersect SB county coastal zone (1000 yards from perimeter)

Open SB county coastal zone shapefile (rough polygon)

In [2]:
fp = os.path.join(os.getcwd(),'coastal_buffer','coastal_buffer.shp')
geom = gpd.read_file(fp)

In [3]:
# geometry = geometry that NAIP scenes must intersect
# year = year of the NAIP scenes you want
# returns a list of the items of NAIP scenes from year that intersect geometry
def query_geom(geom, year):

    date_range = str(year)+'-01-01/'+str(year)+'-12-31'

    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1")

    search = catalog.search(
        collections=["naip"], 
        intersects=geom, 
        datetime=date_range)
    
    items =list(search.get_items()) 
    if len(items)==0:
        return None
    return items


In [4]:
year = 2018
coast_scenes = query_geom(geom.geometry[0],year)
print(len(coast_scenes))
#coast_scenes

28


## Check finer resolution buffer

In [5]:
fp = os.path.join(os.getcwd(),'sb_coast','sb_coast.shp')
coast = gpd.read_file(fp)
#coast

In [6]:
for scene in coast_scenes:
    s_corners = scene.properties['proj:bbox']
    s_box = box(s_corners[0], s_corners[1], s_corners[2], s_corners[3])
    coast_match = coast.to_crs(scene.properties['proj:epsg']).geometry[0]
    
    if s_box.intersects(coast_match) == False:
        coast_scenes.remove(scene)

In [7]:
len(coast_scenes)

25

In [8]:
# extract itemids and create dataframe
itemids = []
for item in coast_scenes:
    itemids.append(item.id)
    
itemids_df = pd.DataFrame(itemids, columns = ['itemid'])
itemids_df['year'] = year
print(itemids_df.shape)

(25, 2)


In [9]:
# save item ids
itemids_df.to_csv(os.path.join(os.getcwd(),'temp','coastal_scenes_ids_'+str(year)+'.csv'), index=False)

# Group NAIP scenes by date

In [10]:
def make_bbox(item):
    c = item.properties['proj:bbox']
    return box(c[0],c[1],c[2],c[3])

In [11]:
# make dataframe with collected scenes and relevant info
df = pd.DataFrame(coast_scenes, columns = ['item'])
df['date']=df.item.apply(lambda item : item.properties['datetime'])
df['crs']=df.item.apply(lambda item : item.properties['proj:epsg'])
df['bbox']=df.item.apply(make_bbox)
df.shape

(25, 4)

In [12]:
def group_flight(df,date):
    same_date = df[df['date']==date]
    
    crs_list = same_date.crs.unique()
    polygons = []
    for crs in crs_list:
        same_crs = same_date[same_date['crs']==crs]
        area = shapely.ops.unary_union(same_crs.bbox)
        gdf = gpd.GeoDataFrame({'geometry':[area]}, 
                               crs='EPSG:'+str(crs))
        gdf.to_crs('EPSG:4326',inplace=True)
        polygons.append(gdf.geometry[0])

        flight = shapely.ops.unary_union(polygons)

    return flight

# --------------------------------------------------------------------------------------------------------

def flight_paths(df):
    dates = df.date.unique()
    flights = []
    for date in dates:
        flights.append(group_flight(df,date))
    gdf = gpd.GeoDataFrame({'date':dates, 'geometry':flights},
                     crs = 'EPSG:4326')
    return gdf

In [13]:
flights = flight_paths(df)

In [14]:
flights

,date,geometry
0,2018-09-13T00:00:00Z,"MULTIPOLYGON (((-120.50354 34.43618, -120.5016..."
1,2018-07-24T00:00:00Z,"MULTIPOLYGON (((-119.50345 34.37357, -119.5640..."
2,2018-07-22T00:00:00Z,"MULTIPOLYGON (((-119.81405 34.37217, -119.8140..."
3,2018-07-15T00:00:00Z,"POLYGON ((-119.99846 34.43456, -119.99607 34.5..."


In [15]:
# save if needed
flights.to_file(os.path.join(os.getcwd(),'temp','coastal_flights_'+str(year)+'.shp'))